In [1]:
sc

## 1) Write a Flume Configuration file for Extracting Twitter Data About covid-19

agent1.sources =source1
agent1.sinks = sink1
agent1.channels = channel1

agent1.sources.source1.channels = channel1
agent1.sinks.sink1.channel = channel1

agent1.sources.source1.type = spooldir
agent1.sources.source1.spoolDir = /home/vandana/spooldir

agent1.sinks.sink1.type = hdfs
agent1.sinks.sink1.hdfs.path = /flume
agent1.sinks.sink1.hdfs.filePrefix = events
agent1.sinks.sink1.hdfs.fileSuffix = .log
agent1.sinks.sink1.hdfs.inUsePrefix = _
agent1.sinks.sink1.hdfs.fileType = DataStream

agent1.channels.channel1.type = file





flume-ng agent --conf-file spool-to-hdfs.properties --name agent1 -Dflume.root.logger=WARN,console

## 2) Write a sqoop job for extracting data from mysql and put In Hive Warehouse

sqoop import --connect jdbc:mysql://localhost:3306/sqoopdemo --username=root --password=password --table=pet --target-dir /vandana  --m 1

## 5) Write a Spark Streaming Python Program to Count Error in real Time.

from pyspark import SparkContext
from pyspark.streaming import StreamingContext

sc = SparkContext("local[2]","StreamingErrorCount")
ssc = StreamingContext(sc,10)

ssc.checkpoint("/streaming")
data = ssc.socketTextStream("localhost", 9999)
count = data.flatMap(lambda x:x.split(" ")).filter(lambda word:"Error" in word).map(lambda word:(word,1)).reduceByKey(lambda x,y:x+y)

count.pprint()
ssc.start()
ssc.awaitTermination()

In [2]:
fpth="hdfs:///spark/rdd/flights.csv"

In [4]:
fdata= sc.textFile(fpth)

## 3) Write Pyspark code to perform actions like (take.collect,count,first)

In [5]:
fdata.take(10)

[u'"FL_DATE","AIRLINE_ID","FL_NUM","ORIGIN","DEST","DEP_TIME","DEP_DELAY","ARR_TIME","ARR_DELAY","AIR_TIME","DISTANCE",',
 u'2016-01-01,19790,"1248","DTW","LAX","1935",0.00,"2120",-24.00,249.00,1979.00,',
 u'2016-01-01,19790,"1251","ATL","GRR","2130",5.00,"2319",-2.00,92.00,640.00,',
 u'2016-01-01,19790,"1254","LAX","ATL","2256",1.00,"0547",-13.00,207.00,1947.00,',
 u'2016-01-01,19790,"1255","SLC","ATL","1700",4.00,"2213",-16.00,173.00,1590.00,',
 u'2016-01-01,19790,"1256","BZN","MSP","1012",72.00,"1420",124.00,121.00,874.00,',
 u'2016-01-01,19790,"1257","ATL","BNA","1356",83.00,"1402",83.00,38.00,214.00,',
 u'2016-01-01,19790,"1257","BNA","ATL","1446",86.00,"1644",74.00,37.00,214.00,',
 u'2016-01-01,19790,"1258","ATL","JAX","0946",1.00,"1053",3.00,45.00,270.00,',
 u'2016-01-01,19790,"1258","JAX","ATL","1144",-1.00,"1247",-15.00,43.00,270.00,']

In [6]:
fdata.collect()

[u'"FL_DATE","AIRLINE_ID","FL_NUM","ORIGIN","DEST","DEP_TIME","DEP_DELAY","ARR_TIME","ARR_DELAY","AIR_TIME","DISTANCE",',
 u'2016-01-01,19790,"1248","DTW","LAX","1935",0.00,"2120",-24.00,249.00,1979.00,',
 u'2016-01-01,19790,"1251","ATL","GRR","2130",5.00,"2319",-2.00,92.00,640.00,',
 u'2016-01-01,19790,"1254","LAX","ATL","2256",1.00,"0547",-13.00,207.00,1947.00,',
 u'2016-01-01,19790,"1255","SLC","ATL","1700",4.00,"2213",-16.00,173.00,1590.00,',
 u'2016-01-01,19790,"1256","BZN","MSP","1012",72.00,"1420",124.00,121.00,874.00,',
 u'2016-01-01,19790,"1257","ATL","BNA","1356",83.00,"1402",83.00,38.00,214.00,',
 u'2016-01-01,19790,"1257","BNA","ATL","1446",86.00,"1644",74.00,37.00,214.00,',
 u'2016-01-01,19790,"1258","ATL","JAX","0946",1.00,"1053",3.00,45.00,270.00,',
 u'2016-01-01,19790,"1258","JAX","ATL","1144",-1.00,"1247",-15.00,43.00,270.00,',
 u'2016-01-01,19790,"1259","ATL","OKC","2107",-3.00,"2224",-12.00,116.00,761.00,',
 u'2016-01-01,19790,"1260","MSP","SMF","1113",-2.00,"1236",-

In [7]:
fdata.count()

445828

In [8]:
fdata.first()

u'"FL_DATE","AIRLINE_ID","FL_NUM","ORIGIN","DEST","DEP_TIME","DEP_DELAY","ARR_TIME","ARR_DELAY","AIR_TIME","DISTANCE",'

## 8) Write a pySpark Code to Clean Data By Using User Defined Function, Handle the same namedTuple Function and Padded all blank Fields with “00.00”.

In [9]:
blanks = fdata.map(lambda x:','.join(x or '00.00' for x in x.split(',')))
Final=blanks.map(lambda x:x.replace(',""' , ',"0000"') )

In [10]:
import csv 
from StringIO import StringIO

def notHeader(row):
    return "AIRLINE_ID" not in row

def split(line):
    reader = csv.reader(StringIO(line))
    return reader.next()


In [11]:
from datetime import datetime 
from collections import namedtuple

fields   = ('date', 'airline', 'flightnum', 'origin', 'dest', 'dep',
            'dep_delay', 'arv', 'arv_delay', 'airtime', 'distance')

Flight   = namedtuple('Flight', fields, verbose=False)

DATE_FMT = '%Y-%m-%d'

TIME_FMT = '%H%M%S' # %S is added to remove error "unconverted data remains 0 " in strptime()


def parse(row):
    row[0]  = datetime.strptime(row[0], DATE_FMT).date()
    row[5]  = datetime.strptime(row[5], TIME_FMT).time()
    row[6]  = float(row[6])
    row[7]  = datetime.strptime(row[7], TIME_FMT).time()
    row[8]  = float(row[8])
    row[9]  = float(row[9])
    row[10] = float(row[10])
    return Flight(*row[:11])

In [12]:
f=Final.filter(notHeader).map(split)

In [13]:
flights=f.map(parse)

## 6) Write a Spark RDD code to get average delay in flights.

In [16]:
totalDelay=flights.map(lambda x:x.dep_delay).reduce(lambda x,y:x+y)
c=flights.count()
print "average delay is:"+str(totalDelay/c)

average delay is:7.59297664789


## 7) Write a pySpark Code to Calculate Total No. of Flights Departed From Airport “LAX” on ‘2016-01-01’

In [17]:
output=flights.filter((lambda x: (x.origin=="LAX") and (x.date == datetime.strptime("2016-01-01", "%Y-%m-%d").date()))).count()
print output

523


## 4) Write a Spark sql code to convert json data and select query where screen_user.name-realDonaldTrump


In [19]:
tPth="hdfs:///spark/cache-0.json"

In [20]:
import json
Data=sc.textFile(tPth).map(lambda x:json.loads(x))

In [21]:
from pyspark.sql import SQLContext, Row

In [22]:
sqlC=SQLContext(sc)

In [23]:
sqlC

In [27]:
table=sqlC.read.json(tPth)

In [28]:
table.registerTempTable("twitterTable")

In [29]:
sqlC.sql("Select text, user.screen_name from twitterTable where user.screen_name='realDonaldTrump' limit 10").collect()

[Row(text=u'Obama asked a 7 yr old for his birth certificate. He\'s "in your face" because the Republicans dropped the ball. (cont) http://t.co/FufZD79U', screen_name=u'realDonaldTrump'),
 Row(text=u'Obama is taunting the Republicans on the birther issue. They should call his bluff &amp; demand the REAL facts. He (cont) http://t.co/NWmVp06e', screen_name=u'realDonaldTrump'),
 Row(text=u'"President Obama is the greatest hoax ever perpetrated on the American people"\n--Clint Eastwood', screen_name=u'realDonaldTrump')]